# 🗄️ Day 1: SQL for AI + Production Vector Databases

**🎯 Goal:** Master SQL for data workflows and production-grade vector databases

**⏱️ Time:** 120-150 minutes

**🌟 Why This Matters for AI (2025):**
- **SQL appears in 26% of AI/ML job postings** - critical for data engineering
- Vector databases are THE foundation of RAG systems (ChatGPT, Claude, etc.)
- Every AI company uses: Pinecone, Weaviate, or Chroma in production
- RAG is the #1 AI application pattern - you MUST know vector DBs
- SQL + Vector DBs = Complete data stack for modern AI

**What You'll Build Today:**
1. Master SQL queries for AI data workflows
2. Build production RAG with Pinecone (cloud vector DB)
3. Use Weaviate for semantic search with hybrid search
4. Deploy local RAG with ChromaDB
5. Compare vector DB performance and choose the right one

---

## 📊 Part 1: SQL for AI/ML Engineers

**Why AI Engineers Need SQL:**

✅ **Data Collection**: Query training data from databases  
✅ **Feature Engineering**: JOIN tables to create features  
✅ **Model Monitoring**: Query prediction logs, track metrics  
✅ **A/B Testing**: Analyze experiment results  
✅ **Production**: Most companies store data in SQL databases  

### 🎯 Essential SQL for AI:

**1. SELECT - Retrieve Data**
```sql
SELECT user_id, prediction, confidence
FROM ml_predictions
WHERE confidence > 0.8;
```

**2. JOIN - Combine Tables**
```sql
SELECT u.user_id, u.age, p.prediction
FROM users u
JOIN predictions p ON u.user_id = p.user_id;
```

**3. GROUP BY - Aggregate Metrics**
```sql
SELECT model_version, AVG(accuracy) as avg_accuracy
FROM model_metrics
GROUP BY model_version;
```

**4. Window Functions - Running Metrics**
```sql
SELECT date, accuracy,
       AVG(accuracy) OVER (ORDER BY date ROWS 7 PRECEDING) as rolling_avg
FROM daily_metrics;
```

In [ ]:
# Install SQL libraries
import sys
!{sys.executable} -m pip install sqlite3 pandas sqlalchemy --quiet

print("✅ SQL libraries installed!")

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

# Create sample AI/ML database
conn = sqlite3.connect(':memory:')  # In-memory database for demo
cursor = conn.cursor()

# Create tables for ML workflow
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    age INTEGER,
    country TEXT,
    signup_date DATE
)
''')

cursor.execute('''
CREATE TABLE model_predictions (
    prediction_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    model_version TEXT,
    prediction TEXT,
    confidence REAL,
    timestamp DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
)
''')

cursor.execute('''
CREATE TABLE model_metrics (
    metric_id INTEGER PRIMARY KEY,
    model_version TEXT,
    date DATE,
    accuracy REAL,
    precision_score REAL,
    recall REAL
)
''')

# Insert sample data
users_data = [
    (1, 25, 'USA', '2024-01-15'),
    (2, 34, 'UK', '2024-01-20'),
    (3, 28, 'Canada', '2024-02-01'),
    (4, 45, 'USA', '2024-02-10'),
    (5, 31, 'Germany', '2024-02-15')
]
cursor.executemany('INSERT INTO users VALUES (?,?,?,?)', users_data)

predictions_data = [
    (1, 1, 'v1.0', 'high_value', 0.92, '2024-03-01 10:00:00'),
    (2, 1, 'v1.0', 'high_value', 0.88, '2024-03-02 11:00:00'),
    (3, 2, 'v1.0', 'low_value', 0.76, '2024-03-01 12:00:00'),
    (4, 3, 'v2.0', 'high_value', 0.95, '2024-03-03 09:00:00'),
    (5, 4, 'v2.0', 'medium_value', 0.82, '2024-03-04 14:00:00'),
    (6, 5, 'v2.0', 'high_value', 0.91, '2024-03-05 10:30:00')
]
cursor.executemany('INSERT INTO model_predictions VALUES (?,?,?,?,?,?)', predictions_data)

metrics_data = [
    (1, 'v1.0', '2024-03-01', 0.85, 0.82, 0.88),
    (2, 'v1.0', '2024-03-02', 0.86, 0.83, 0.89),
    (3, 'v2.0', '2024-03-03', 0.91, 0.89, 0.93),
    (4, 'v2.0', '2024-03-04', 0.92, 0.90, 0.94),
    (5, 'v2.0', '2024-03-05', 0.93, 0.91, 0.95)
]
cursor.executemany('INSERT INTO model_metrics VALUES (?,?,?,?,?,?)', metrics_data)

conn.commit()
print("✅ Sample ML database created with users, predictions, and metrics!")

In [ ]:
# Example 1: Simple SELECT - Get high-confidence predictions
query = '''
SELECT prediction_id, user_id, prediction, confidence
FROM model_predictions
WHERE confidence > 0.9
ORDER BY confidence DESC;
'''

df = pd.read_sql_query(query, conn)
print("🎯 High-Confidence Predictions (>90%):")
print(df)
print(f"\n💡 Found {len(df)} predictions with >90% confidence")

In [ ]:
# Example 2: JOIN - Combine user data with predictions
query = '''
SELECT 
    u.user_id,
    u.age,
    u.country,
    p.prediction,
    p.confidence,
    p.model_version
FROM users u
JOIN model_predictions p ON u.user_id = p.user_id
WHERE p.confidence > 0.85;
'''

df = pd.read_sql_query(query, conn)
print("🔗 User Demographics + Predictions (JOINed):")
print(df)
print(f"\n💡 This is how you create features by combining tables!")

In [ ]:
# Example 3: GROUP BY - Compare model versions
query = '''
SELECT 
    model_version,
    COUNT(*) as num_predictions,
    AVG(confidence) as avg_confidence,
    MIN(confidence) as min_confidence,
    MAX(confidence) as max_confidence
FROM model_predictions
GROUP BY model_version;
'''

df = pd.read_sql_query(query, conn)
print("📊 Model Version Comparison:")
print(df)
print(f"\n💡 v2.0 has higher average confidence - it's performing better!")

In [ ]:
# Example 4: Window Functions - Rolling metrics
query = '''
SELECT 
    date,
    model_version,
    accuracy,
    AVG(accuracy) OVER (
        PARTITION BY model_version 
        ORDER BY date 
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) as rolling_avg_accuracy
FROM model_metrics
ORDER BY model_version, date;
'''

df = pd.read_sql_query(query, conn)
print("📈 Rolling Accuracy (3-day window):")
print(df)
print(f"\n💡 Window functions track metrics over time - critical for monitoring!")

conn.close()

## 🗄️ Part 2: Production Vector Databases

**Vector Database Landscape (2025):**

| Database | Type | Best For | Pricing |
|----------|------|----------|----------|
| **Pinecone** | Cloud | Production RAG, scalability | Paid (free tier) |
| **Weaviate** | Self-hosted/Cloud | Hybrid search, flexibility | Open-source |
| **ChromaDB** | Local/Embedded | Development, small projects | Free |
| **Qdrant** | Self-hosted/Cloud | High performance, filtering | Open-source |
| **Milvus** | Self-hosted | Enterprise scale | Open-source |
| **FAISS** | Library | Research, prototyping | Free |

**Today's Focus: Pinecone + Weaviate + ChromaDB** (most popular in industry)

---

### 🌲 Pinecone: Cloud-Native Vector Database

**Why Pinecone:**
- Fully managed (no infrastructure)
- Scales to billions of vectors
- Fast similarity search (<100ms)
- Used by: OpenAI, Notion, Zapier

**Use Case:** Production RAG for customer support chatbot

In [ ]:
# Install Pinecone
!{sys.executable} -m pip install pinecone-client sentence-transformers --quiet

print("✅ Pinecone installed!")
print("\n⚠️ To use Pinecone:")
print("   1. Sign up at https://www.pinecone.io (free tier available)")
print("   2. Get your API key")
print("   3. Uncomment the code below and add your key")

In [ ]:
# Pinecone RAG System (Production-Ready)

# UNCOMMENT AND ADD YOUR API KEY TO RUN:
'''
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import time

# Initialize Pinecone
pc = Pinecone(api_key="YOUR_API_KEY_HERE")

# Create index (vector database)
index_name = "ai-docs-rag"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Matches our embedding model
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"✅ Created index: {index_name}")

# Connect to index
index = pc.Index(index_name)

# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample documents (company knowledge base)
documents = [
    {"id": "doc1", "text": "Our AI chatbot supports 24/7 customer service with 95% accuracy."},
    {"id": "doc2", "text": "To reset your password, click 'Forgot Password' on the login page."},
    {"id": "doc3", "text": "Our premium plan includes unlimited API calls and priority support."},
    {"id": "doc4", "text": "Shipping typically takes 3-5 business days for domestic orders."},
    {"id": "doc5", "text": "We offer a 30-day money-back guarantee on all purchases."},
]

# Embed and upload documents
vectors = []
for doc in documents:
    embedding = embedding_model.encode(doc["text"]).tolist()
    vectors.append({
        "id": doc["id"],
        "values": embedding,
        "metadata": {"text": doc["text"]}
    })

# Upsert to Pinecone
index.upsert(vectors=vectors)
print(f"✅ Uploaded {len(vectors)} documents to Pinecone")

# Wait for index to be ready
time.sleep(2)

# Query the system
query = "How do I reset my password?"
query_embedding = embedding_model.encode(query).tolist()

# Search
results = index.query(
    vector=query_embedding,
    top_k=2,
    include_metadata=True
)

print(f"\n🔍 Query: '{query}'")
print("\n📄 Retrieved Documents:")
for match in results['matches']:
    print(f"  [Score: {match['score']:.4f}] {match['metadata']['text']}")

print("\n🎉 Pinecone RAG system working! This is production-ready.")
'''

print("👆 Uncomment the code above and add your Pinecone API key to run!")
print("\n💡 Pinecone is used by companies for production RAG at scale")

### 🔷 Weaviate: Hybrid Search Vector Database

**Why Weaviate:**
- Combines vector search + keyword search (hybrid)
- Built-in ML models (no separate embedding step)
- GraphQL API
- Self-hostable (you own your data)

**Use Case:** Semantic search with filtering

In [ ]:
# Install Weaviate
!{sys.executable} -m pip install weaviate-client --quiet

print("✅ Weaviate installed!")
print("\n💡 For this demo, we'll use Weaviate Cloud (free tier)")
print("   Or run locally with: docker run -p 8080:8080 semitechnologies/weaviate")

In [ ]:
# Weaviate Hybrid Search Demo

# UNCOMMENT TO RUN (requires Weaviate instance):
'''
import weaviate
from weaviate.classes.init import Auth

# Connect to Weaviate (local or cloud)
client = weaviate.connect_to_local()  # For local Docker instance
# OR for cloud: client = weaviate.connect_to_wcs(cluster_url="...", auth_credentials=Auth.api_key("..."))

# Create collection (schema)
if not client.collections.exists("Article"):
    client.collections.create(
        name="Article",
        vectorizer_config=weaviate.Configure.Vectorizer.text2vec_transformers(),
        properties=[
            weaviate.Property(name="title", data_type=weaviate.DataType.TEXT),
            weaviate.Property(name="content", data_type=weaviate.DataType.TEXT),
            weaviate.Property(name="category", data_type=weaviate.DataType.TEXT),
        ]
    )

# Get collection
articles = client.collections.get("Article")

# Add documents
articles.data.insert_many([
    {
        "title": "Understanding Transformers",
        "content": "Transformers use self-attention to process sequences in parallel.",
        "category": "AI"
    },
    {
        "title": "RAG Systems Guide",
        "content": "Retrieval-Augmented Generation combines search with LLMs.",
        "category": "AI"
    },
    {
        "title": "Python Best Practices",
        "content": "Use type hints and docstrings for better code quality.",
        "category": "Programming"
    }
])

print("✅ Documents added to Weaviate")

# Hybrid search (combines vector + keyword search)
response = articles.query.hybrid(
    query="how do transformers work?",
    limit=2
)

print("\n🔍 Hybrid Search Results:")
for item in response.objects:
    print(f"  Title: {item.properties['title']}")
    print(f"  Content: {item.properties['content']}")
    print(f"  Category: {item.properties['category']}")
    print()

client.close()
'''

print("👆 Uncomment to run with Weaviate!")
print("\n💡 Hybrid search = semantic (meaning) + keyword (exact match)")
print("   Best for: Production search where you need both precision and recall")

### 🎨 ChromaDB: Local Vector Database

**Why ChromaDB:**
- Runs locally (no API keys needed!)
- Simple Python API
- Perfect for development and prototyping
- Persistent storage

**Use Case:** Local RAG system for personal documents

**This one we can run right now!** ✅

In [ ]:
# Install ChromaDB
!{sys.executable} -m pip install chromadb --quiet

print("✅ ChromaDB installed!")

In [ ]:
import chromadb
from chromadb.config import Settings

# Initialize ChromaDB (persistent)
client = chromadb.Client()

# Create or get collection
collection = client.create_collection(
    name="ai_knowledge_base",
    metadata={"description": "AI/ML knowledge articles"}
)

print("✅ ChromaDB collection created!")
print(f"   Collection: {collection.name}")

In [ ]:
# Add documents to ChromaDB
documents = [
    "GPT-4 is a large language model developed by OpenAI with over 1 trillion parameters.",
    "RAG systems retrieve relevant documents before generating answers, reducing hallucinations.",
    "LoRA (Low-Rank Adaptation) enables efficient fine-tuning by updating only small adapter layers.",
    "Vector databases like Pinecone and Weaviate are essential for semantic search at scale.",
    "Prompt engineering is the art of crafting effective prompts to get better LLM outputs.",
    "Fine-tuning adapts pre-trained models to specific domains using task-specific data.",
    "Transformers use self-attention mechanisms to process sequences in parallel, unlike RNNs.",
    "BERT is an encoder-only transformer designed for understanding tasks like classification.",
    "LangChain provides a framework for building LLM applications with chains and agents.",
    "Embeddings convert text into dense vectors that capture semantic meaning."
]

# ChromaDB auto-generates embeddings!
collection.add(
    documents=documents,
    ids=[f"doc{i}" for i in range(len(documents))],
    metadatas=[{"topic": "AI", "index": i} for i in range(len(documents))]
)

print(f"✅ Added {len(documents)} documents to ChromaDB")
print("\n💡 ChromaDB automatically created embeddings using its default model!")

In [ ]:
# Query ChromaDB
query = "How can I make LLMs more accurate?"

results = collection.query(
    query_texts=[query],
    n_results=3
)

print(f"🔍 Query: '{query}'\n")
print("📄 Retrieved Documents:\n")

for i, (doc, distance, metadata) in enumerate(zip(
    results['documents'][0],
    results['distances'][0],
    results['metadatas'][0]
), 1):
    print(f"{i}. [Similarity: {1-distance:.3f}] {doc}")
    print(f"   Metadata: {metadata}\n")

print("💡 ChromaDB found relevant docs about RAG and fine-tuning!")

In [ ]:
# Advanced: Filtering with metadata
results_filtered = collection.query(
    query_texts=["Tell me about transformers"],
    n_results=2,
    where={"topic": "AI"}  # Metadata filter
)

print("🔍 Query with Filter: 'Tell me about transformers' (topic=AI)\n")
print("📄 Results:\n")

for i, doc in enumerate(results_filtered['documents'][0], 1):
    print(f"{i}. {doc}\n")

print("💡 Metadata filtering lets you search within specific subsets!")

## 🏆 Vector Database Comparison

### When to Use Each:

**🌲 Pinecone:**
- ✅ Production applications at scale
- ✅ Need managed service (no infrastructure)
- ✅ Willing to pay for reliability
- **Example:** Customer support chatbot for 10,000+ users

**🔷 Weaviate:**
- ✅ Need hybrid search (semantic + keyword)
- ✅ Want self-hosted option (data privacy)
- ✅ Complex filtering requirements
- **Example:** Enterprise search across internal documents

**🎨 ChromaDB:**
- ✅ Development and prototyping
- ✅ Small-scale projects (<100K vectors)
- ✅ Need simple, local setup
- **Example:** Personal knowledge base, learning projects

### Performance Comparison:

| Feature | Pinecone | Weaviate | ChromaDB |
|---------|----------|----------|----------|
| **Scale** | Billions | Millions | Thousands-Millions |
| **Speed** | <100ms | <200ms | Varies |
| **Setup** | API key only | Docker/Cloud | `pip install` |
| **Cost** | $70+/mo | Free (self-host) | Free |
| **Hybrid Search** | ❌ | ✅ | ❌ |
| **GraphQL** | ❌ | ✅ | ❌ |
| **Learning Curve** | Easy | Medium | Easy |

**💡 Pro Tip:** Start with ChromaDB for development, then move to Pinecone/Weaviate for production!

## 🎯 Complete RAG System with ChromaDB

Let's build a production-quality RAG system using everything we learned!

In [ ]:
from transformers import pipeline

# Initialize LLM for generation (using GPT-2 for demo)
generator = pipeline('text-generation', model='gpt2')

def rag_query(question, top_k=3):
    """
    Complete RAG pipeline with ChromaDB
    """
    # Step 1: Retrieve relevant documents
    results = collection.query(
        query_texts=[question],
        n_results=top_k
    )
    
    retrieved_docs = results['documents'][0]
    
    # Step 2: Build context
    context = "\n\n".join([
        f"[{i+1}] {doc}" 
        for i, doc in enumerate(retrieved_docs)
    ])
    
    # Step 3: Create prompt
    prompt = f"""Answer the question using the provided context.

Context:
{context}

Question: {question}

Answer:"""
    
    # Step 4: Generate answer
    response = generator(
        prompt,
        max_length=len(prompt.split()) + 50,
        num_return_sequences=1,
        temperature=0.7
    )
    
    answer = response[0]['generated_text'][len(prompt):].strip()
    
    return {
        'answer': answer,
        'sources': retrieved_docs,
        'context': context
    }

# Test the RAG system
questions = [
    "What is RAG and how does it help?",
    "How can I fine-tune models efficiently?",
    "What are transformers?"
]

print("🤖 Production RAG System with ChromaDB\n")
print("="*80)

for question in questions:
    print(f"\n❓ Question: {question}\n")
    
    result = rag_query(question)
    
    print("📚 Retrieved Sources:")
    for i, source in enumerate(result['sources'], 1):
        print(f"   [{i}] {source}")
    
    print(f"\n💬 Generated Answer:")
    print(f"   {result['answer'][:200]}...")  # Truncate for display
    
    print("\n" + "-"*80)

print("\n🎉 Complete RAG system working!")
print("\n💡 In production, you'd use:")
print("   - GPT-4 or Claude for better generation")
print("   - Pinecone/Weaviate for scale")
print("   - Re-ranking for better retrieval")
print("   - Streaming for better UX")

## 🎓 Key Takeaways

### SQL for AI:
✅ Essential for data collection and feature engineering  
✅ JOINs combine data from multiple tables  
✅ GROUP BY aggregates metrics for model evaluation  
✅ Window functions track metrics over time  
✅ 26% of AI jobs require SQL - master it!  

### Vector Databases:
✅ Foundation of RAG systems (the #1 AI pattern)  
✅ **Pinecone:** Production-ready, fully managed, scales to billions  
✅ **Weaviate:** Hybrid search, self-hostable, GraphQL API  
✅ **ChromaDB:** Local development, simple API, perfect for learning  
✅ All major AI companies use vector DBs in production  

### Best Practices:
✅ Start with ChromaDB for development  
✅ Use Pinecone/Weaviate for production at scale  
✅ Implement metadata filtering for better search  
✅ Combine with LLMs for complete RAG systems  
✅ Monitor and optimize embedding quality  

---

**You now have production-ready skills for:**
- 🗄️ Querying AI/ML data with SQL
- 🔍 Building semantic search with vector databases
- 🤖 Deploying production RAG systems
- 📊 Choosing the right vector DB for your use case

**Next:** Day 2 - Advanced RAG Techniques + LLM Agents + LoRA/QLoRA! 🚀